In [1]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import os 

def create_folder(folder_name):
    """建立資料夾"""
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    return folder_name

def dataframe_to_point(df, lon_col, lat_col, crs="EPSG:4326", target_crs="EPSG:3826"):
    '''
    Parameters:
    df (dataframe) : 含經緯度座標欄位的dataframe
    lon_col (str) : 緯度欄位
    Lat_col (str) : 經度欄位
    crs (str) : 目前經緯度座標的座標系統，常用的為4326(WGS84)、3826(TWD97)
    target_crs：目標轉換的座標系統
    '''

    from shapely.geometry import Point
    import pandas as pd
    import geopandas as gpd
    # Create Point geometries from the longitude and latitude columns
    geometry = [Point(xy) for xy in zip(df[lon_col], df[lat_col])]
    # Create a GeoDataFrame with the original CRS
    gdf = gpd.GeoDataFrame(df, geometry=geometry, crs=crs)
    # Convert the GeoDataFrame to the target CRS
    gdf = gdf.to_crs(epsg=target_crs.split(":")[1])
    return gdf

def downloadfile(downloadpath, url):
    """
    下載 XML 檔案內容。
    :param downloadpath: 儲存 XML 檔案的路徑
    :param url: XML 檔案的下載 URL
    :return: XML 檔案內容
    """
    response = requests.get(url)
    if response.status_code == 200:
        with open(downloadpath, 'wb') as file:
            file.write(response.content)
        print(f"XML 檔案已下載至 {downloadpath}")
        return response.content
    else:
        raise Exception(f"無法下載檔案，HTTP 狀態碼: {response.status_code}")

def parse_vd_xml(xml_content):
    """
    解析 XML 檔案為 DataFrame，提取所有相關欄位。
    :param xml_content: XML 檔案內容
    :return: 轉換後的 DataFrame
    """
    namespace = {'ns': 'http://traffic.transportdata.tw/standard/traffic/schema/'}
    root = ET.fromstring(xml_content)

    data = []
    for vd in root.findall('.//ns:VDs/ns:VD', namespace):
        # 基本資料
        vdid = vd.find('ns:VDID', namespace).text
        sub_authority = vd.find('ns:SubAuthorityCode', namespace).text
        bi_directional = vd.find('ns:BiDirectional', namespace).text
        vd_type = vd.find('ns:VDType', namespace).text
        location_type = vd.find('ns:LocationType', namespace).text
        detection_type = vd.find('ns:DetectionType', namespace).text
        position_lon = vd.find('ns:PositionLon', namespace).text
        position_lat = vd.find('ns:PositionLat', namespace).text
        road_id = vd.find('ns:RoadID', namespace).text if vd.find('ns:RoadID', namespace) is not None else None
        road_name = vd.find('ns:RoadName', namespace).text if vd.find('ns:RoadName', namespace) is not None else None
        road_class = vd.find('ns:RoadClass', namespace).text if vd.find('ns:RoadClass', namespace) is not None else None
        location_mile = vd.find('ns:LocationMile', namespace).text if vd.find('ns:LocationMile', namespace) is not None else None

        # 道路區間
        road_section_start = vd.find('./ns:RoadSection/ns:Start', namespace).text if vd.find('./ns:RoadSection/ns:Start', namespace) is not None else None
        road_section_end = vd.find('./ns:RoadSection/ns:End', namespace).text if vd.find('./ns:RoadSection/ns:End', namespace) is not None else None

        # DetectionLinks 資料
        for link in vd.findall('./ns:DetectionLinks/ns:DetectionLink', namespace):
            link_id = link.find('ns:LinkID', namespace).text
            bearing = link.find('ns:Bearing', namespace).text if link.find('ns:Bearing', namespace) is not None else None
            road_direction = link.find('ns:RoadDirection', namespace).text if link.find('ns:RoadDirection', namespace) is not None else None
            lane_num = link.find('ns:LaneNum', namespace).text if link.find('ns:LaneNum', namespace) is not None else None
            actual_lane_num = link.find('ns:ActualLaneNum', namespace).text if link.find('ns:ActualLaneNum', namespace) is not None else None

            # 將資料加入清單
            data.append({
                "VDID": vdid,
                "SubAuthorityCode": sub_authority,
                "BiDirectional": bi_directional,
                "VDType": vd_type,
                "LocationType": location_type,
                "DetectionType": detection_type,
                "PositionLon": position_lon,
                "PositionLat": position_lat,
                "RoadID": road_id,
                "RoadName": road_name,
                "RoadClass": road_class,
                "LocationMile": location_mile,
                "RoadSectionStart": road_section_start,
                "RoadSectionEnd": road_section_end,
                "LinkID": link_id,
                "Bearing": bearing,
                "RoadDirection": road_direction,
                "LaneNum": lane_num,
                "ActualLaneNum": actual_lane_num,
            })

    return pd.DataFrame(data)

def download_and_parce_VD(downloadfolder, url):
    download_path = os.path.join(downloadfolder, 'VD.xml')
    csv_output_path = os.path.join(downloadfolder, 'VD.csv')
    
    xml_content = downloadfile(download_path, url = url)
    # 解析 XML
    df = parse_vd_xml(xml_content)
    # 儲存為 CSV
    df.to_csv(csv_output_path, index=False, encoding="big5")
    return df
    


/Users/zhangkaijie/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
urls = {"VD" : "https://tisvcloud.freeway.gov.tw/history/motc20/VD.xml",
        "ETagPair" : "https://tisvcloud.freeway.gov.tw/history/motc20/ETagPair.xml",
        "ETag" : "https://tisvcloud.freeway.gov.tw/history/motc20/ETag.xml",
        "CCTV" : "https://tisvcloud.freeway.gov.tw/history/motc20/CCTV.xml"}

downloadfolder = create_folder(os.path.join(os.getcwd(),'高公局靜態資料清單'))
shpfolder = create_folder(os.path.join(downloadfolder, 'shp'))

VD = download_and_parce_VD(downloadfolder=downloadfolder, url = urls['VD']) 
VD = dataframe_to_point(VD, lon_col='PositionLon', lat_col='PositionLat')
VD.to_file(os.path.join(shpfolder,'VD.shp'))

XML 檔案已下載至 /Users/zhangkaijie/Desktop/Work/THI/THI-git/THI-GIS-Tool/高公局靜態資料清單/VD.xml


/var/folders/1p/d63vtkgd60d4sbpl791031gw0000gn/T/ipykernel_2570/2602947974.py:11: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  VD.to_file(os.path.join(shpfolder,'VD.shp'))
/Users/zhangkaijie/Library/Python/3.9/lib/python/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'SubAuthorityCode' to 'SubAuthori'
  ogr_write(
/Users/zhangkaijie/Library/Python/3.9/lib/python/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'BiDirectional' to 'BiDirectio'
  ogr_write(
/Users/zhangkaijie/Library/Python/3.9/lib/python/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'LocationType' to 'LocationTy'
  ogr_write(
/Users/zhangkaijie/Library/Python/3.9/lib/python/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'DetectionType' to 'DetectionT'
  ogr_write(
/Users/zhangkaijie/Library/Python/3.9/lib/python/site-package

In [3]:
VD

,VDID,SubAuthorityCode,BiDirectional,VDType,LocationType,DetectionType,PositionLon,PositionLat,RoadID,RoadName,RoadClass,LocationMile,RoadSectionStart,RoadSectionEnd,LinkID,Bearing,RoadDirection,LaneNum,ActualLaneNum,geometry
0,VD-N3-S-15.700-M-LOOP,NFB-NR,0,1,5,1,121.62179,25.036867,000030,國道3號,0,15K+700,None,None,0000300001550A,SW,S,5,5,POINT (312745.534 2770005.119)
1,VD-N2-E-7.900-M-LOOP,NFB-NR,0,1,5,1,121.266,25.0267,000020,國道2號,0,7K+900,None,None,0000200000700H,E,E,4,4,POINT (276844.296 2768761.27)
2,VD-N3-S-152.800-M-RS,NFB-CR,0,1,5,1,120.68301,24.430344,000030,國道3號,0,152K+800,通霄交流道,苑裡交流道,0000300015200K,S,S,3,3,POINT (217856.975 2702720.611)
3,VD-N1-N-266.672-M-RS,NFB-SR,0,2,1,1,120.379875,23.4759,000010,國道1號,0,266K+672,水上交流道,嘉義交流道,0000100126600Q,NE,N,3,3,POINT (186653.762 2597119.084)
4,VD-N1-N-374.270-M-RS,NFB-SR,0,1,5,1,120.3179944,22.5783373,000010,國道1號,0,374K+270,高雄端,漁港路交流道,0000101176090E,NE,N,2,2,POINT (179869.6 2497752.673)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3250,VD-N3-S-176-I-WS-1X-南下入口2,NFB-CR,0,1,5,2,120.59075,24.248,000030,國道3號,0,176K+000,沙鹿交流道,龍井交流道,0000301034060B,SE,S,2,2,POINT (208442.196 2682549.816)
3251,VD-N1-N-276.010-M-Loop,NFB-SR,0,1,5,1,120.35527,23.399374,000010,國道1號,0,276K+010,新營服務區,嘉義系統交流道,0000100127600D,N,N,3,3,POINT (184102.339 2588655.347)
3252,VD-N3-N-186.072-M-RS,NFB-CR,0,1,5,1,120.54896,24.178804,000030,國道3號,0,186K+072,和美交流道,龍井交流道,0000300118600B,NE,N,3,3,POINT (204173.779 2674899.268)
3253,VD-N1-S-54.410-M-LOOP,NFB-NR,0,1,5,1,121.257484,25.00648,000010,國道1號,0,54K+410,機場系統交流道,中壢服務區,0000100005400H,SW,S,4,4,POINT (275989.127 2766519.986)


In [ ]:
# 主程式
if __name__ == "__main__":
    downloadfolder = create_folder(os.path.join(os.getcwd(),'高公局靜態資料清單'))
    download_path = os.path.join(downloadfolder, 'VD.xml')
    csv_output_path = os.path.join(downloadfolder, 'VD.csv')
    # 下載 XML
    xml_content = downloadfile(download_path)
    # 解析 XML
    df = parse_vd_xml(xml_content)
    # 儲存為 CSV
    df.to_csv(csv_output_path, index=False, encoding="big5")
    print(f"DataFrame 已儲存為 {csv_output_path}")
    print(df.columns)